## Prerequisites
- A [WRITER AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)
- Basic familiarity with Python and [AWS Strands](https://dev.writer.com/home/integrations/strands)

## Installation and setup

First, we need to install the WRITER SDK MCP package globally using npm. This provides the MCP server that will allow us to interact with WRITER's AI tools and services.



In [ ]:
!npm install -g writer-sdk-mcp

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os

os.environ["WRITER_API_KEY"] = getpass.getpass()

## Starting the MCP server

Now we'll start the WRITER MCP server on port 3000. This server acts as a bridge between our Python code and WRITER's services, providing access to tools like content generation, knowledge graphs, file processing, and more. The server runs in the background using subprocess.


In [ ]:
import subprocess

# Start MCP server on port 3000 (can change if needed)
server = subprocess.Popen(
    ["npx", "-y", "writer-sdk-mcp", "--transport=http", "--port=3000"],
    env=os.environ,
)
print("✅ WRITER MCP Server running at http://localhost:3000")


## Example: Building an Assistant with WRITER + LangChain SDK

For this example, we’ll create a simple assistant using the [**WRITER LangChain SDK integration**](https://dev.writer.com/home/integrations/langchain), which connects the **WRITER MCP SDK** (Python package) to the **LangChain framework**.  

This setup allows your LangChain agents to call WRITER’s MCP tools directly through the [**LangChain MCP integration**](https://docs.langchain.com/oss/python/langchain/mcp).

### Python dependencies

We'll install **LangChain** with the **WRITER MCP integration**, which gives us:
- **`langchain`** – The AI agent framework for building and orchestrating agents.  
- **`langchain-mcp-adapters`** – Integration to connect LangChain agents to WRITER’s MCP tools.  
- **`langchain-writer`** – Access WRITER models like Palmyra X5 from LangChain agents.  

Let's start with the installations:


In [ ]:
%pip install -U langchain

In [ ]:
%pip install langchain-mcp-adapters

## System prompt configuration

Here we define the system prompt that guides the Strands Agent’s behavior.
This prompt instructs the agent to:
- Act autonomously in reading, writing, and analyzing information.
- Use WRITER’s MCP-connected tools for file processing, editing, and knowledge graph interactions, etc.
- Provide clear, human-readable explanations and summaries of its work.


In [ ]:
system_prompt = """
You are a research and writing assistant built on the WRITER SDK and connected to the Model Control Protocol (MCP).

Your goals:
1. Help users research technical, creative, or analytical topics.
2. Produce clear, accurate, and well-structured text.
3. When using tools, explain what you’re doing and why, unless instructed otherwise.
4. Be concise and factual in tone.

Always prioritize correctness, clarity, and tool-assisted reasoning.
"""

## Creating the Agent

This code creates a `LangChainWriterAgent` class that:

- Connects to a local MCP server running on port 3000.
- Uses WRITER’s Palmyra X5 model for natural language processing and reasoning.
- Connects the model to the `MultiServerMCPClient`, enabling access to available tools.
- Provides an interactive chat interface for research and development tasks using LangChain.

The agent can help you research technical, creative, or analytical topics, and can also invoke WRITER MCP tools directly.


In [ ]:
import asyncio
import subprocess
import os
import time
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langchain_writer import ChatWriter

# Make sure your WRITER_API_KEY is set
os.environ["WRITER_API_KEY"] = os.getenv("WRITER_API_KEY", "YOUR_API_KEY_HERE")

class LangChainWriterAgent:
    """Lightweight research agent using WRITER's MCP tools via LangChain."""

    def __init__(self, system_prompt: str, server, mcp_url="http://localhost:3000"):

        # Connect to MCP
        self.client = MultiServerMCPClient(
            {
                "research": {
                    "transport": "streamable_http",
                    "url": mcp_url,
                }
            }
        )
        self.model = ChatWriter(model="palmyra-x5")
        self.system_prompt = system_prompt
        self.agent = None
        self.server = server

    async def init_agent(self):
        """Initialize LangChain agent with tools from MCP."""
        tools = await self.client.get_tools()
        self.agent = create_agent(self.model, tools, system_prompt=self.system_prompt)

    async def chat(self, query: str) -> str:
        """Send a query to the agent and stream results."""
        if not self.agent:
            await self.init_agent()

        final_response = ""
        async for chunk in self.agent.astream(
            {"messages": [{"role": "user", "content": query}]},
            stream_mode="updates"
        ):
            for step, data in chunk.items():
                text = data['messages'][-1].content_blocks[0]['text']
                print(text)
                final_response += text
        return final_response

    def stop(self):
        """Stop the MCP server."""
        self.server.terminate()
        self.server.wait()
        print("🛑 WRITER MCP Server stopped.")


### Using the Agent

Initialize the `LangChainWriterAgent` to connect the WRITER MCP package with LangChain. This lets your local agent use WRITER’s MCP tools through the Palmyra model interface.

**Try this query:**

> “Tell me about no-code apps—what they are, how they work (general description, do not share names of no-code agents), and which model versions I can use.”


In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

async def main():
    system_prompt = """
You are a research and writing assistant built on the WRITER SDK and connected to the Model Control Protocol (MCP).

Your goals:
1. Help users research technical, creative, or analytical topics.
2. Produce clear, accurate, and well-structured text.
3. When using tools, explain what you’re doing and why.
4. Be concise and factual in tone.
"""
    agent = LangChainWriterAgent(system_prompt=system_prompt, server=server)
    try:
        query = "Tell me about no-code apps—what they are, how they work (general, do not share names of no-code agents), and which model versions I can use."
        response = await agent.chat(query)
        print("\n\n✅ Final Response:\n", response)
    finally:
        agent.stop()

asyncio.run(main())
